In [1]:
# Imports
import numpy as np
import pandas as pd
import pulp

from warnings import filterwarnings
filterwarnings("ignore")


vehicles_df = pd.read_csv('"../data/Master_data/vehicle_master_data_4.csv"')
distances_df = pd.read_csv('../data/Master_data/distance_master_data_4.csv')


OSError: [Errno 22] Invalid argument: '"../data/Master_data/vehicle_master_data_4.csv"'

In [ ]:
input_df = pd.read_csv("../data/Input_data/Input_1_Small.csv")
vehicles_df
# input_df

,Transportation modes,Vehicle_type,Capacity (metric tons),Co2e (g/km),Avg Speed (km/hr),Cost per km,Minimum Quantity Allowed (MQA)
0,Air,CARGO PLANE,100,551.584454,500,10.00,90.0
1,Rail,GOODS RAIL,8000,97.565914,50,4.00,4000.0
2,Rail,MIXED RAIL,5000,107.090591,90,5.50,2000.0
3,Road,HGV DROPSIDE LORRY,24,156.598994,45,1.30,10.0
4,Road,4WD PICK UP,18,155.605476,42,1.20,10.0
5,Road,VAN,12,211.365144,70,1.40,5.0
6,Road,PICK UP,10,209.320194,55,1.34,3.0
7,Road,HGV (Large),36,155.398594,34,1.23,20.0
8,Road,CAR DERIVED VAN,2,187.850299,65,1.41,0.5
9,Road,ELECTRIC,7,151.765792,58,1.52,2.0


In [ ]:
input_df

,From,To,Quantity (MT),Time (hrs),Cost
0,Delhi,Mumbai,9000,300,50000
1,Delhi,Kolkata,26500,300,30000
2,Delhi,Chennai,13,1000,15600


In [ ]:
# Create a dictionary 'vehicles' containing information about vehicles categorized by their 'Transportation modes'
vehicles = {}
for mode in vehicles_df['Transportation modes']:
    # Create a list to store information about vehicles for the current mode
    vehicles_list = []
    
    # Iterate over the rows that belong to the current 'Transportation mode' in the 'vehicles_df' DataFrame
    for j in range(len(vehicles_df[vehicles_df['Transportation modes'] == mode])):
        # Create a dictionary 'vehicle_info' for each row of 'vehicles_df' belonging to the current mode 'mode'
        vehicle_info = {}
        
        # Iterate over the columns (except for 'Transportation modes') and store the information in 'vehicle_info'
        for column in vehicles_df[vehicles_df['Transportation modes'] == mode].columns[1:]:
            vehicle_info[column] = vehicles_df[column][j]
        
        # Add the 'vehicle_info' dictionary to the 'vehicles_list' for the current mode
        vehicles_list.append(vehicle_info)
    
    # Add the list of vehicles for the current mode 'mode' to the 'vehicles' dictionary with the mode as the key
    vehicles[mode] = vehicles_list

# Create a dictionary 'distances' containing information about distances between 'Source' and 'Destination'
distances = {}
for j in range(len(distances_df)):
    # Create a dictionary 'distance_info' for each row of 'distances_df' for the current source and destination
    distance_info = {}
    
    # Iterate over the columns (starting from the third column) and store the distance information in 'distance_info'
    for column in distances_df.columns[2:]:
        distance_info[column] = distances_df[column][j]
    
    # Add the 'distance_info' dictionary to the 'distances' dictionary with the source and destination as the key
    distances[(distances_df['Source'][j], distances_df['Destination'][j])] = distance_info

# Perform a cross join between 'distances_df' and 'vehicles_df' to create a new DataFrame 'df'
df = distances_df.join(vehicles_df, how="cross")

# Create a new column 'decisions' in 'df' containing a string concatenation of various columns
df['decisions'] = df['Source'] + ', ' + df['Destination'] + ', ' + df['Transportation modes'] + ', ' + df['Vehicle_type']


In [ ]:
df

,Source,Destination,Air Distance (km),Rail Distance (km),Road Distance (km),Transportation modes,Vehicle_type,Capacity (metric tons),Co2e (g/km),Avg Speed (km/hr),Cost per km,Minimum Quantity Allowed (MQA),decisions
0,Mumbai,Kolkata,1655.79,1723.03,2071.24,Air,CARGO PLANE,100,551.584454,500,10.00,90.0,"Mumbai, Kolkata, Air, CARGO PLANE"
1,Mumbai,Kolkata,1655.79,1723.03,2071.24,Rail,GOODS RAIL,8000,97.565914,50,4.00,4000.0,"Mumbai, Kolkata, Rail, GOODS RAIL"
2,Mumbai,Kolkata,1655.79,1723.03,2071.24,Rail,MIXED RAIL,5000,107.090591,90,5.50,2000.0,"Mumbai, Kolkata, Rail, MIXED RAIL"
3,Mumbai,Kolkata,1655.79,1723.03,2071.24,Road,HGV DROPSIDE LORRY,24,156.598994,45,1.30,10.0,"Mumbai, Kolkata, Road, HGV DROPSIDE LORRY"
4,Mumbai,Kolkata,1655.79,1723.03,2071.24,Road,4WD PICK UP,18,155.605476,42,1.20,10.0,"Mumbai, Kolkata, Road, 4WD PICK UP"
...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,Chennai,Bangalore,290.95,303.61,365.20,Road,HGV (Small),15,190.411547,52,1.43,3.0,"Chennai, Bangalore, Road, HGV (Small)"
376,Chennai,Bangalore,290.95,303.61,365.20,Road,LARGE VAN,8,145.588808,60,1.44,2.0,"Chennai, Bangalore, Road, LARGE VAN"
377,Chennai,Bangalore,290.95,303.61,365.20,Road,MOTOR CARAVAN,13,125.195841,50,1.47,5.0,"Chennai, Bangalore, Road, MOTOR CARAVAN"
378,Chennai,Bangalore,290.95,303.61,365.20,Road,HGV LOADER,4,186.902781,28,1.37,1.0,"Chennai, Bangalore, Road, HGV LOADER"


In [ ]:
# Create a dictionary 'master_availability' to store the initial availability of different vehicle types
# The initial availability for all vehicle types is set to 10
master_availability = dict(zip(df['Vehicle_type'].unique(), [2] * len(df['Vehicle_type'].unique())))
master_availability["HGV (Large)"]=3
# Create a dictionary 'master_demand' to store the initial demand for vehicles between different sources and destinations
# The initial demand for all unique source-destination combinations is set to 0
master_demand = {}

# Get the unique combinations of 'Source' and 'Destination' from the DataFrame 'df'
unique_source_destination = np.array(df[['Source', 'Destination']].drop_duplicates())

# Iterate over the unique combinations of 'Source' and 'Destination'
for source_dest in unique_source_destination:
    # Use the combination as a key in the 'master_demand' dictionary and set the initial demand to 0
    master_demand[tuple(source_dest)] = 0


master_availability


{'CARGO PLANE': 2,
 'GOODS RAIL': 2,
 'MIXED RAIL': 2,
 'HGV DROPSIDE LORRY': 2,
 '4WD PICK UP': 2,
 'VAN': 2,
 'PICK  UP': 2,
 'HGV (Large)': 3,
 'CAR DERIVED VAN': 2,
 'ELECTRIC': 2,
 'TIPPER': 2,
 'TIPPER (HGV)': 2,
 'PANEL VAN': 2,
 'ESTATE': 2,
 'HGV (Small)': 2,
 'LARGE VAN': 2,
 'MOTOR CARAVAN': 2,
 'HGV LOADER': 2,
 '4x4': 2}

In [ ]:
# Create a dictionary 'demand' to store the demand for vehicles between different "From" and "To" locations
demand = {}

# Extract the unique combinations of "From" and "To" locations from the DataFrame 'input_df'
unique_source_destination = np.array(input_df[['From', 'To']])

# Extract the corresponding demand values (quantity in metric tons) from the DataFrame 'input_df'
demand_values = input_df['Quantity (MT)']

# Pair each unique combination of "From" and "To" locations with the corresponding demand value
for source_dest, demand_value in zip(unique_source_destination, demand_values):
    # Use the combination as a key in the 'demand' dictionary and set the demand value as the value
    demand[tuple(source_dest)] = demand_value

# Iterate over the keys of the 'master_demand' dictionary to check for missing keys in 'demand'
for key in master_demand.keys():
    # If the key does not exist in 'demand', set its value to 0 (indicating no demand)
    if key not in demand:
        demand[key] = 0


demand

{('Delhi', 'Mumbai'): 9000,
 ('Delhi', 'Kolkata'): 26500,
 ('Delhi', 'Chennai'): 13,
 ('Mumbai', 'Kolkata'): 0,
 ('Mumbai', 'Delhi'): 0,
 ('Mumbai', 'Bangalore'): 0,
 ('Mumbai', 'Chennai'): 0,
 ('Kolkata', 'Mumbai'): 0,
 ('Kolkata', 'Delhi'): 0,
 ('Kolkata', 'Bangalore'): 0,
 ('Kolkata', 'Chennai'): 0,
 ('Delhi', 'Bangalore'): 0,
 ('Bangalore', 'Mumbai'): 0,
 ('Bangalore', 'Kolkata'): 0,
 ('Bangalore', 'Delhi'): 0,
 ('Bangalore', 'Chennai'): 0,
 ('Chennai', 'Mumbai'): 0,
 ('Chennai', 'Kolkata'): 0,
 ('Chennai', 'Delhi'): 0,
 ('Chennai', 'Bangalore'): 0}

In [ ]:
%%time

##########################################################
#------------------- Model Definition -------------------#
##########################################################
# Create a LP minimization problem
model = pulp.LpProblem("Co2e Optimization", pulp.LpMinimize)
            
############################################################
#------------------- Decision Variables -------------------#
############################################################

# Create a dictionary to store both binary decision variables with prefixes "Choice_Decision_" and "Return_Count_"
choice_decision = pulp.LpVariable.dicts("Choice_Decision_", 
                                        (tuple(df['decisions'][i].split(", ")) + (j,) 
                                          for i in range(len(df)) 
                                          for j in range(master_availability[df['decisions'][i].split(", ")[-1]]) ),
                                        lowBound=0, cat="Binary")

# Add the "Return_Count_" decision variables with the same format
return_count_decision = pulp.LpVariable.dicts("Return_Count_", 
                                              (tuple(df['decisions'][i].split(", ")) + (j,) 
                                                for i in range(len(df)) 
                                                for j in range(master_availability[df['decisions'][i].split(", ")[-1]]) ),
                                              lowBound=0, cat="Integer")


############################################################
#------------------- Objective Function -------------------#
############################################################

# Create an empty list to store the contributions to the objective function
obj_tup = []

# Iterate through each combination of decision variables
for (s, d, m, v, c) in choice_decision.keys():
    # Retrieve the binary decision variable for the current combination
    decision_var = choice_decision[(s, d, m, v, c)]
    return_count_key = tuple([s, d, m, v, c])
    decision_return_var = return_count_decision[return_count_key]

    # Retrieve the distance and CO2 emissions for the current combination from the DataFrame 'df'
    distance_km = df[(df['Source'] == s) & (df['Destination'] == d) & 
                     (df['Transportation modes'] == m) & (df['Vehicle_type'] == v)][m + ' Distance (km)'].item()
    # print(distance_km)
    co2_emissions_g_per_km = df[(df['Source'] == s) & (df['Destination'] == d) & 
                                (df['Transportation modes'] == m) & (df['Vehicle_type'] == v)]['Co2e (g/km)'].item()

    # print(decision_var,decision_return_var)

    # Calculate the contribution to the objective function for the current combination and add it to 'obj_tup'
    contribution = 2*decision_var * distance_km * (co2_emissions_g_per_km / 1000)
    
    contribution1 = 2*decision_return_var * distance_km * (co2_emissions_g_per_km / 1000)
    # print(contribution)
    obj_tup.append(contribution)
    obj_tup.append(contribution1)

# Sum up all the contributions to form the objective function and add it to the optimization model
model += pulp.lpSum(obj_tup)

# obj_tup

# #####################################################
# #------------------- Constraints -------------------#
# #####################################################

# #========== Demand Constraint ==========
(s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
col = []

for (s, d, m, v, c) in choice_decision.keys():

    if(s0, d0) == (s, d):
        col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
        col.append(return_count_decision[(s,d,m,v,c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
    else:
        if demand[(s0, d0)] == 0:
            model += (pulp.lpSum(col)==demand[(s0, d0)])
            (s0, d0) = (s, d)
            col=[]
            col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
            col.append(return_count_decision[(s,d,m,v,c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
        elif demand[(s0, d0)] >= 0:
            model += (pulp.lpSum(col)>=demand[(s0, d0)])
            (s0, d0) = (s, d)
            col=[]
            col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
            col.append(return_count_decision[(s,d,m,v,c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())

if demand[(s0, d0)] >= 0:
    model += (pulp.lpSum(col)>=demand[(s0, d0)])
elif demand[(s0, d0)] >= 0:
    model += (pulp.lpSum(col)>=demand[(s0, d0)])

# #========== Minimum Quantity Allowed Constraint ==========
(s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
col1 = []

for (s, d, m, v, c) in choice_decision.keys():

    if(s0, d0) == (s, d):
        col1.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Minimum Quantity Allowed (MQA)'].item())
        col1.append(return_count_decision[(s,d,m,v,c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Minimum Quantity Allowed (MQA)'].item())
    else:
        if demand[(s0, d0)] == 0:
            model += (pulp.lpSum(col1)<=demand[(s0, d0)])
            (s0, d0) = (s, d)
            col1=[]
            col1.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Minimum Quantity Allowed (MQA)'].item())
            col1.append(return_count_decision[(s,d,m,v,c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Minimum Quantity Allowed (MQA)'].item())
        if demand[(s0, d0)] >= 0:
            model += (pulp.lpSum(col1)<=demand[(s0, d0)])
            (s0, d0) = (s, d)
            col1=[]
            col1.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Minimum Quantity Allowed (MQA)'].item())
            col1.append(return_count_decision[(s,d,m,v,c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Minimum Quantity Allowed (MQA)'].item())

if demand[(s0, d0)] >= 0:
    model += (pulp.lpSum(col1)<=demand[(s0, d0)])
if demand[(s0, d0)] >= 0:
    model += (pulp.lpSum(col1)<=demand[(s0, d0)])


#========== Cost Constraint ==========
(s0, d0, m0, v0, c0) = list(choice_decision.keys())[0]
col = []

for (s, d, m, v, c) in choice_decision.keys():
    if((s0, d0) == (s, d)) & input_df[(input_df['From']==s) & (input_df['To']==d)].shape[0]:
        col.append(return_count_decision[(s,d,m,v,c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)][m+' Distance (km)'].item()*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Cost per km'].item())
        col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)][m+' Distance (km)'].item()*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Cost per km'].item())
    elif (s0, d0) != (s, d):
        if input_df[(input_df['From']==s0) & (input_df['To']==d0)].shape[0]:
            model += (pulp.lpSum(col)<=input_df[(input_df['From']==s0) & (input_df['To']==d0)]['Cost'])
            (s0, d0) = (s, d)
            col=[]
            col.append(choice_decision[(s, d, m, v, c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
            col.append(return_count_decision[(s,d,m,v,c)]*df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Capacity (metric tons)'].item())
        else:
            (s0, d0) = (s, d)
            col=[]

if input_df[(input_df['From']==s) & (input_df['To']==d)].shape[0]:
    model += (pulp.lpSum(col)<=input_df[(input_df['From']==s0) & (input_df['To']==d0)]['Cost'])


#========== Other Constraint ==========

for (s, d, m, v, c) in choice_decision.keys():
    return_var= return_count_decision[(s,d,m,v,c)]
    choice_var= choice_decision[(s,d,m,v,c)]
    model += return_var <= 10000000*choice_var




for (s, d, m, v, c) in choice_decision.keys():
    if m != "Road":
        return_var= return_count_decision[(s,d,m,v,c)]
    
        model += return_var ==0

#========== Time Constraint ==========
for (s, d, m, v, c) in choice_decision.keys():
    if input_df[(input_df['From']==s) & (input_df['To']==d)].shape[0]:
        exp1 = df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)][m+' Distance (km)'].item()/df[(df['Source']==s) & (df['Destination']==d) & (df['Transportation modes']==m) & (df['Vehicle_type']==v)]['Avg Speed (km/hr)'].item()
        exp= (choice_decision[(s, d, m, v, c)]*exp1)+(return_count_decision[(s,d,m,v,c)]*2.5*exp1)
        # print(exp,"eeeee")
        model += pulp.LpConstraint(e=pulp.LpAffineExpression(exp), sense=pulp.LpConstraintLE, rhs = input_df[(input_df['From']==s) & (input_df['To']==d)]['Time (hrs)'].item())






    






# return_count_decision

CPU times: total: 7.28 s
Wall time: 7.39 s


In [ ]:
%%time
###############################################
#------------------- Solve -------------------#
###############################################

model.solve()
print(pulp.LpStatus[model.status])


Optimal
CPU times: total: 78.1 ms
Wall time: 172 ms


In [ ]:
# optimal_combinations = []
# for v in model.variables():
#     if v.varValue > 0:  # Consider non-zero values to get the selected combinations
#         optimal_combinations.append((v.name, v.varValue))

# # Sort the combinations based on CO2 emissions (assuming 'CO2e' is the CO2 emissions variable name)
# # optimal_combinations.sort(key=lambda x: float(x[0].split('_')[-1]), reverse=False)

# # Print the combinations in ascending order of CO2 emissions
# for combination, count in optimal_combinations:
#     print(f"Combination: {combination}, Count: {count}, CO2 Emissions: {(combination.split('_')[-1])}")

In [ ]:
model.variables

<bound method LpProblem.variables of Co2e_Optimization:
MINIMIZE
320.96699366622*Choice_Decision__('Bangalore',_'Chennai',_'Air',_'CARGO_PLANE',_0) + 320.96699366622*Choice_Decision__('Bangalore',_'Chennai',_'Air',_'CARGO_PLANE',_1) + 59.243974493390404*Choice_Decision__('Bangalore',_'Chennai',_'Rail',_'GOODS_RAIL',_0) + 59.243974493390404*Choice_Decision__('Bangalore',_'Chennai',_'Rail',_'GOODS_RAIL',_1) + 65.027548727742*Choice_Decision__('Bangalore',_'Chennai',_'Rail',_'MIXED_RAIL',_0) + 65.027548727742*Choice_Decision__('Bangalore',_'Chennai',_'Rail',_'MIXED_RAIL',_1) + 113.65423967040002*Choice_Decision__('Bangalore',_'Chennai',_'Road',_'4WD_PICK_UP',_0) + 113.65423967040002*Choice_Decision__('Bangalore',_'Chennai',_'Road',_'4WD_PICK_UP',_1) + 116.24451211648*Choice_Decision__('Bangalore',_'Chennai',_'Road',_'4x4',_0) + 116.24451211648*Choice_Decision__('Bangalore',_'Chennai',_'Road',_'4x4',_1) + 137.2058583896*Choice_Decision__('Bangalore',_'Chennai',_'Road',_'CAR_DERIVED_VAN',_0

In [ ]:
#######################################################################
#------------------- Mappings Necessary for Output -------------------#
#######################################################################

mt_choice = []
mt_return =[]
for v in model.variables():
    if  v.varValue != 0 :
        if v.name.split('__(')[0]=="Choice_Decision":
            mt_choice.append(v.name.split('__(')[1].rstrip(')').split(',_') + [v.varValue,])
        else:
            mt_return.append(v.name.split('__(')[1].rstrip(')').split(',_') + [v.varValue+1,])

mt_choice_output = pd.DataFrame(mt_choice, columns=['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Vechile Chosen'])
mt_choice_output_df = mt_choice_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
mt_choice_output_df['S_D_M'] = mt_choice_output_df['Source'] + '_' + mt_choice_output_df['Destination'] + '_' + mt_choice_output_df['Transportation mode']


mt_return_output = pd.DataFrame(mt_return, columns=['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'Number of Trips'])
mt_return_output_df = mt_return_output.applymap(lambda x: x.strip("'") if isinstance(x, str) else x)
mt_return_output_df['S_D_M'] = mt_return_output_df['Source'] + '_' + mt_return_output_df['Destination'] + '_' + mt_return_output_df['Transportation mode']

# mt_return_output_df

merged_df = mt_choice_output_df.merge(mt_return_output_df, on=['Source', 'Destination', 'Transportation mode', 'Vehicle type', 'Vehicle number', 'S_D_M'], how='left')

# Fill NaN values in the 'Return Count' column with 0 (if needed)
merged_df['Number of Trips'].fillna(1, inplace=True)
count_idx = merged_df.columns.get_loc('Vechile Chosen')
merged_df.insert(count_idx + 1, 'Number of Trips', merged_df.pop('Number of Trips'))


merged_df

     



,Source,Destination,Transportation mode,Vehicle type,Vehicle number,Vechile Chosen,Number of Trips,S_D_M
0,Delhi,Chennai,Road,MOTOR_CARAVAN,0,1.0,1.0,Delhi_Chennai_Road
1,Delhi,Kolkata,Air,CARGO_PLANE,0,1.0,1.0,Delhi_Kolkata_Air
2,Delhi,Kolkata,Air,CARGO_PLANE,1,1.0,1.0,Delhi_Kolkata_Air
3,Delhi,Kolkata,Rail,GOODS_RAIL,0,1.0,1.0,Delhi_Kolkata_Rail
4,Delhi,Kolkata,Rail,GOODS_RAIL,1,1.0,1.0,Delhi_Kolkata_Rail
5,Delhi,Kolkata,Rail,MIXED_RAIL,0,1.0,1.0,Delhi_Kolkata_Rail
6,Delhi,Kolkata,Rail,MIXED_RAIL,1,1.0,1.0,Delhi_Kolkata_Rail
7,Delhi,Kolkata,Road,HGV_(Large),0,1.0,2.0,Delhi_Kolkata_Road
8,Delhi,Kolkata,Road,HGV_(Large),1,1.0,3.0,Delhi_Kolkata_Road
9,Delhi,Kolkata,Road,HGV_(Large),2,1.0,3.0,Delhi_Kolkata_Road


In [ ]:

merged_df.sort_values(by=['Source', 'Destination'], inplace=True)

merged_df.reset_index(drop=True, inplace=True)
merged_df


,Source,Destination,Transportation mode,Vehicle type,Vehicle number,Vechile Chosen,Number of Trips,S_D_M
0,Delhi,Chennai,Road,MOTOR_CARAVAN,0,1.0,1.0,Delhi_Chennai_Road
1,Delhi,Kolkata,Air,CARGO_PLANE,0,1.0,1.0,Delhi_Kolkata_Air
2,Delhi,Kolkata,Air,CARGO_PLANE,1,1.0,1.0,Delhi_Kolkata_Air
3,Delhi,Kolkata,Rail,GOODS_RAIL,0,1.0,1.0,Delhi_Kolkata_Rail
4,Delhi,Kolkata,Rail,GOODS_RAIL,1,1.0,1.0,Delhi_Kolkata_Rail
5,Delhi,Kolkata,Rail,MIXED_RAIL,0,1.0,1.0,Delhi_Kolkata_Rail
6,Delhi,Kolkata,Rail,MIXED_RAIL,1,1.0,1.0,Delhi_Kolkata_Rail
7,Delhi,Kolkata,Road,HGV_(Large),0,1.0,2.0,Delhi_Kolkata_Road
8,Delhi,Kolkata,Road,HGV_(Large),1,1.0,3.0,Delhi_Kolkata_Road
9,Delhi,Kolkata,Road,HGV_(Large),2,1.0,3.0,Delhi_Kolkata_Road


In [ ]:
source_destination_table = merged_df[(merged_df['Source'] == 'Delhi') & (merged_df['Destination'] == 'Kolkata')].copy()
source_destination_table


,Source,Destination,Transportation mode,Vehicle type,Vehicle number,Vechile Chosen,Number of Trips,S_D_M
1,Delhi,Kolkata,Air,CARGO_PLANE,0,1.0,1.0,Delhi_Kolkata_Air
2,Delhi,Kolkata,Air,CARGO_PLANE,1,1.0,1.0,Delhi_Kolkata_Air
3,Delhi,Kolkata,Rail,GOODS_RAIL,0,1.0,1.0,Delhi_Kolkata_Rail
4,Delhi,Kolkata,Rail,GOODS_RAIL,1,1.0,1.0,Delhi_Kolkata_Rail
5,Delhi,Kolkata,Rail,MIXED_RAIL,0,1.0,1.0,Delhi_Kolkata_Rail
6,Delhi,Kolkata,Rail,MIXED_RAIL,1,1.0,1.0,Delhi_Kolkata_Rail
7,Delhi,Kolkata,Road,HGV_(Large),0,1.0,2.0,Delhi_Kolkata_Road
8,Delhi,Kolkata,Road,HGV_(Large),1,1.0,3.0,Delhi_Kolkata_Road
9,Delhi,Kolkata,Road,HGV_(Large),2,1.0,3.0,Delhi_Kolkata_Road
10,Delhi,Kolkata,Road,MOTOR_CARAVAN,0,1.0,1.0,Delhi_Kolkata_Road


In [ ]:
%%time
#========== New Constraint ==========
co2_emission_by_1st_iteration=0
for index, row in merged_df.iterrows():
    s = row['Source']
    d = row['Destination']
    m = row['Transportation mode']
    v = row['Vehicle type']
    v = v.replace("_", " ")
    n_t=row['Number of Trips']
    # print(n_t)
    
    distance_km = df[(df['Source'] == s) & (df['Destination'] == d) & 
                     (df['Transportation modes'] == m) & (df['Vehicle_type'] == v)][m + ' Distance (km)'].item()
    co2_emissions_g_per_km = df[(df['Source'] == s) & (df['Destination'] == d) & 
                                (df['Transportation modes'] == m) & (df['Vehicle_type'] == v)]['Co2e (g/km)'].item()
 
    # print(co2_emissions_g_per_km)
    # break
    
    co2_emission_by_1st_iteration+= 2*n_t*distance_km*co2_emissions_g_per_km

# co2_emission_by_1st_iteration= co2_emission_by_1st_iteration


model += pulp.LpConstraint(e=pulp.LpAffineExpression(pulp.lpSum(obj_tup)), sense=pulp.LpConstraintLE, rhs=co2_emission_by_1st_iteration)


# df



CPU times: total: 78.1 ms
Wall time: 78 ms


In [ ]:
%%time
###############################################
#------------------- Solve -------------------#
###############################################

model.solve()
print(pulp.LpStatus[model.status])

Infeasible
CPU times: total: 188 ms
Wall time: 287 ms
